# Flow-source: fracture-borehole intersection

Prototype a method able to find borehole-fracture intersections

## Steps:
1. **Find point and normal defining a borehole**

We can compute global coordinates of any point in any borehole using 
* the borehole root coordinates [columns `x, y z` ], denoted $x_r$
* The direction vector [columns `_trig_x, _trig_y, _trig_z`], $r$
* _variable_: depth (scalar) $d$
To express a point in the borehole, we may write: $x = x_r + d \cdot r$

2. **Find a way to express any point on a plane**

We can express a point in a plane using
* Any point on the plane. $x_p$
* The unit normal vector to the plane, $n$ \
To express a point in the plane, we may write: $n\cdot(x - x_p) = 0 $

3. Compute the intersection of a borehole and a plane
We may combine the information above in the following way. \
Combine the equations above:
$$ n \cdot (x_r + d \cdot r - x_p) = 0 \implies n\cdot(x_r-x_p) + d (n\cdot r)=0$$
Then,
$$ d = \frac{n\cdot(x_p - x_r)}{n\cdot r} $$

In [1]:
import sys
sys.path.extend(['C:\\Users\\Haakon\\OneDrive\\Dokumenter\\FORSKNING\\mastersproject', 
                 'C:\\Users\\Haakon\\OneDrive\\Dokumenter\\FORSKNING\\mastersproject\\src\\mastersproject', 
                 'C:/Users/Haakon/OneDrive/Dokumenter/FORSKNING/mastersproject'])
%load_ext autoreload
%autoreload 2

In [33]:
import GTS as gts
import numpy as np
import pandas as pd

# Global coordinates of all structures 

In [3]:
isc = gts.ISCData()
df = isc.structures
df.head()

,depth,azimuth_struc,dip,aperture,type,borehole,x,y,z,length,...,shearzone,_trig_x,_trig_y,_trig_z,x_swiss,y_swiss,z_swiss,x_gts,y_gts,z_gts
0,1.76,239.29,86.44,1.89,Fracture,SBH3,667468.567,158885.383,1733.960,20.55,...,NaN,-0.174210,-0.980982,0.085591,667468.260391,158883.656472,1734.110640,68.260391,83.656472,34.110640
1,2.11,78.11,21.13,0.00,Fracture,PRP3,667468.390,158892.660,1733.100,32.33,...,NaN,-0.382879,0.821461,-0.422618,667467.582126,158894.393282,1732.208275,67.582126,94.393282,32.208275
2,2.26,252.34,77.18,1.83,Fracture,SBH3,667468.567,158885.383,1733.960,20.55,...,NaN,-0.174210,-0.980982,0.085591,667468.173286,158883.165981,1734.153435,68.173286,83.165981,34.153435
3,2.35,169.30,57.47,334.61,Minor ductile Shear-zone,FBS1,667466.424,158888.882,1732.782,44.80,...,NaN,-0.578330,0.621269,-0.528735,667465.064926,158890.341981,1731.539474,65.064926,90.341981,31.539474
4,2.54,134.77,59.79,10.06,Quartz,GEO3,667470.923,158912.008,1732.416,30.10,...,NaN,-0.671720,-0.000469,-0.740805,667469.216830,158912.006809,1730.534356,69.216830,112.006809,30.534356


# borehole geometry

Compute direction vector for each borehole.

The columns `_trig_x, _trig_y, _trig_z` represent the unit direction vector for the borehole.

In [9]:
borehole_data = isc.borehole_geometry.copy()
borehole_data['depth'] = 0

isc.bh_struc_to_global_coords(data=borehole_data,
                              x='x', y='y', z='z',
                              depth='depth',
                              upward_gradient='upward_gradient',
                              azimuth='azimuth')
borehole_data.head()

,x,y,z,length,diameter,azimuth,upward_gradient,borehole,depth,_trig_x,_trig_y,_trig_z,x_swiss,y_swiss,z_swiss,x_gts,y_gts,z_gts
0,667466.424,158888.882,1732.782,44.80,101,317.05,-31.92,FBS1,0,-0.578330,0.621269,-0.528735,667466.424,158888.882,1732.782,66.424,88.882,32.782
1,667470.633,158905.016,1732.725,47.58,101,253.07,-40.46,FBS2,0,-0.727884,-0.221565,-0.648917,667470.633,158905.016,1732.725,70.633,105.016,32.725
2,667471.317,158926.213,1732.263,44.00,101,219.92,-37.33,FBS3,0,-0.510265,-0.609838,-0.606405,667471.317,158926.213,1732.263,71.317,126.213,32.263
3,667470.546,158859.278,1746.458,18.20,101,258.89,-75.13,SBH1,0,-0.251817,-0.049450,-0.966511,667470.546,158859.278,1746.458,70.546,59.278,46.458
4,667468.567,158885.383,1733.960,20.55,101,190.07,4.91,SBH3,0,-0.174210,-0.980982,0.085591,667468.567,158885.383,1733.960,68.567,85.383,33.960


# Calculate the normal vector for each plane
Also store one coordinate for each plane (e.g. the mean of the regression points).

In [47]:
isc = gts.ISCData(path='windows')
isc.shearzones

results = []
for sz in isc.shearzones:
    point_cloud = isc.get_shearzone(sz=sz, coords='gts')
    N = point_cloud.shape[1]
    centroid = np.sum(point_cloud, axis=1) / N
    normal = gts._fit_normal_to_points(point_cloud)
    data = np.atleast_2d(np.hstack((centroid,normal)))
    columns = ('c_x', 'c_y', 'c_z', 'n_x', 'n_y', 'n_z')
    frame = pd.DataFrame(data=data, columns=columns)
    frame['shearzone'] = sz
    results.append(frame)

df = pd.concat(results, ignore_index=True)
df

,c_x,c_y,c_z,n_x,n_y,n_z,shearzone
0,51.456704,109.806622,20.010900,-0.485314,0.861540,-0.149059,S1_1
1,47.969502,113.118832,16.959744,-0.496146,0.841186,-0.215046,S1_2
2,44.821713,117.088891,14.749656,-0.502727,0.813966,-0.291076,S1_3
3,53.395810,105.063501,22.949580,0.121912,0.940431,-0.317375,S3_1
4,52.491075,108.564742,21.878384,0.146613,0.942768,-0.299489,S3_2


In [48]:
def planes():
    isc = gts.ISCData(path='windows')
    isc.shearzones

    results = []
    for sz in isc.shearzones:
        point_cloud = isc.get_shearzone(sz=sz, coords='gts')
        N = point_cloud.shape[1]
        centroid = np.sum(point_cloud, axis=1) / N
        normal = gts._fit_normal_to_points(point_cloud)
        data = np.atleast_2d(np.hstack((centroid,normal)))
        columns = ('c_x', 'c_y', 'c_z', 'n_x', 'n_y', 'n_z')
        frame = pd.DataFrame(data=data, columns=columns)
        frame['shearzone'] = sz
        results.append(frame)

    df = pd.concat(results, ignore_index=True)
    return df

In [50]:
df = planes()
df

,c_x,c_y,c_z,n_x,n_y,n_z,shearzone
0,51.456704,109.806622,20.010900,-0.485314,0.861540,-0.149059,S1_1
1,47.969502,113.118832,16.959744,-0.496146,0.841186,-0.215046,S1_2
2,44.821713,117.088891,14.749656,-0.502727,0.813966,-0.291076,S1_3
3,53.395810,105.063501,22.949580,0.121912,0.940431,-0.317375,S3_1
4,52.491075,108.564742,21.878384,0.146613,0.942768,-0.299489,S3_2
